In [1]:
!nvidia-smi

Thu Jan  5 12:50:38 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:00:05.0 Off |                  Off |
| N/A   37C    P0    65W / 300W |  25874MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:00:06.0 Off |                  Off |
| N/A   

In [2]:
import os
os.getcwd()

'/home/centos/psw/KSRC'

In [3]:
import os 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import random
from tqdm import tqdm

import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from transformers import TrainingArguments, Trainer
from transformers import AutoModelForSequenceClassification, AutoConfig, AutoTokenizer
from tokenizers import Tokenizer
from transformers import PreTrainedTokenizerFast, BartModel



In [17]:
kakao_train = pd.read_csv('data/with_kakao_train.csv',encoding='utf-8')
kakao_train = kakao_train.drop(['Unnamed: 0',"index"], axis=1)
kakao_train.to_csv('data/with_kakao_train.csv',encoding='utf-8')
kakao_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159475 entries, 0 to 159474
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   premise     159475 non-null  object
 1   hypothesis  159475 non-null  object
 2   label       159475 non-null  object
dtypes: object(3)
memory usage: 3.7+ MB


In [35]:
backtrans_df = pd.read_csv('data/backtrans_df',encoding='utf-8')
backtrans_hyp_df = pd.read_csv('data/backtrans_hyp_df',encoding='utf-8')
backtrans = pd.DataFrame({'premise':backtrans_df['premise'],"hypothesis":backtrans_hyp_df['hyphothesis'],'label':backtrans_df['label']})
backtrans.info()
backtrans.to_csv('data/backtrans.csv',encoding='utf-8')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52996 entries, 0 to 52995
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   premise     52996 non-null  object
 1   hypothesis  52996 non-null  object
 2   label       52996 non-null  object
dtypes: object(3)
memory usage: 1.2+ MB


In [36]:
def bullyong(df):
    df['premise'] = df['premise'].str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 0-9]', '')
    df['hypothesis'] = df['hypothesis'].str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 0-9]', '')
    return df

In [37]:
test = pd.read_csv('data/test_data.csv',encoding='utf-8')

kakao_train = bullyong(kakao_train)
backtrans = bullyong(backtrans)
test = bullyong(test)

/home/centos/anaconda3/envs/test_env/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  
/home/centos/anaconda3/envs/test_env/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until


In [38]:
# 결측치는 없음

print(backtrans.info(), end='\n\n')
print(kakao_train.info(), end='\n\n')
print(test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52996 entries, 0 to 52995
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   premise     52996 non-null  object
 1   hypothesis  52996 non-null  object
 2   label       52996 non-null  object
dtypes: object(3)
memory usage: 1.2+ MB
None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159475 entries, 0 to 159474
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   premise     159475 non-null  object
 1   hypothesis  159475 non-null  object
 2   label       159475 non-null  object
dtypes: object(3)
memory usage: 3.7+ MB
None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1666 entries, 0 to 1665
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   index       1666 non-null   int64 
 1   premise     1666 non-null   object
 2   hypothesis  16

In [39]:
def seed_everything(seed:int = 2023):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  # type: ignore

seed_everything()

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [40]:
MODEL_NAME = 'tunib/electra-ko-base'

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
config = AutoConfig.from_pretrained(MODEL_NAME)

config.num_labels = 3

model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, config=config)

print(model)
print(config)

Some weights of the model checkpoint at tunib/electra-ko-base were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at tunib/electra-ko-base and are newly initialized: ['classifier.out_proj.weight', 'classi

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm

In [41]:
backtrans_dataset = train_test_split(backtrans, shuffle=True)
pd.concat([backtrans_dataset[0], backtrans_dataset[1]], axis = 0)

train_dataset, eval_dataset = train_test_split(kakao_train, test_size=0.2, shuffle=True, stratify=kakao_train['label'])
train_dataset = pd.concat([train_dataset,backtrans],axis=0)
train_dataset = train_test_split(train_dataset, shuffle=True)
train_dataset = pd.concat([train_dataset[0], train_dataset[1]], axis = 0)

print(train_dataset.info(), end='\n\n')
print(eval_dataset.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 180576 entries, 85621 to 17512
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   premise     180576 non-null  object
 1   hypothesis  180576 non-null  object
 2   label       180576 non-null  object
dtypes: object(3)
memory usage: 5.5+ MB
None

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31895 entries, 144111 to 151902
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   premise     31895 non-null  object
 1   hypothesis  31895 non-null  object
 2   label       31895 non-null  object
dtypes: object(3)
memory usage: 996.7+ KB
None


In [44]:
# train test split 및 tokenizing 
# token에 들어가는 문장은 premise와 hypothesis를 concat 한 문장

# train_dataset, eval_dataset = train_test_split(train, test_size=0.2, shuffle=True, stratify=train['label'])

tokenized_train = tokenizer(
    list(train_dataset['premise']),
    list(train_dataset['hypothesis']),
    return_tensors="pt",
    max_length=300, # Max_Length = 190
    padding=True,
    truncation=True,
    add_special_tokens=True
)

tokenized_eval = tokenizer(
    list(eval_dataset['premise']),
    list(eval_dataset['hypothesis']),
    return_tensors="pt",
    max_length=300,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

print(tokenized_train['input_ids'][0])
print(tokenizer.decode(tokenized_train['input_ids'][0]))

tensor([    2,  1312, 14208, 19413,  6155, 18077, 24389, 17911,  6037, 16904,
        11301,     3, 14717,  3190,  6220,  6184,  6296, 12550, 11306, 11774,
         1312, 14208, 18077, 24389, 17911,  6037, 16904, 11301,     3,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0])
[CLS] 두 아이가 방치된 더블 유모차에서 자고 있다 [SEP] 엄마가 식료품점 안에 있는 동안 두 아이가 더블 유모차에서 자고 있다 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]

In [45]:
class BERTDataset(torch.utils.data.Dataset):
    def __init__(self, pair_dataset, label):
        self.pair_dataset = pair_dataset
        self.label = label

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.pair_dataset.items()}
        item['label'] = torch.tensor(self.label[idx])
        
        return item

    def __len__(self):
        return len(self.label)
    
def label_to_num(label):
    label_dict = {"entailment": 0, "contradiction": 1, "neutral": 2, "answer": 3}
    num_label = []

    for v in label:
        num_label.append(label_dict[v])

    return num_label


train_label = label_to_num(train_dataset['label'].values)
eval_label = label_to_num(eval_dataset['label'].values)

In [46]:
train_dataset = BERTDataset(tokenized_train, train_label)
eval_dataset = BERTDataset(tokenized_eval, eval_label)

print(train_dataset.__len__())
print(train_dataset.__getitem__(19997))
print(tokenizer.decode(train_dataset.__getitem__(19997)['input_ids']))

180576
{'input_ids': tensor([    2, 25638, 11298, 12851, 11299, 30813,     3, 27905, 12816, 12679,
        11327,     3,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0]), 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [47]:
def compute_metrics(pred):
  """ validation을 위한 metrics function """
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  probs = pred.predictions

  # calculate accuracy using sklearn's function
  acc = accuracy_score(labels, preds) # 리더보드 평가에는 포함되지 않습니다.

  return {
      'accuracy': acc,
  }

In [48]:
training_ars = TrainingArguments(
    output_dir='result/pure_test_electra/',
    num_train_epochs=7,
    per_device_train_batch_size=32,
    save_total_limit=5,
    save_steps=500,
    evaluation_strategy='steps',
    eval_steps = 500,
    load_best_model_at_end = True,
)

trainer = Trainer(
    model=model,
    args=training_ars,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [49]:
trainer.train()
model.save_pretrained('result/pure_test_electra/best_model')

/home/centos/anaconda3/envs/test_env/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 180576
  Num Epochs = 7
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 19754
  Number of trainable parameters = 110619651
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: sangmi. Use `wandb login --relogin` to force relogin


You're using a ElectraTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/centos/anaconda3/envs/test_env/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:61: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss,Accuracy
500,0.655300,0.495499,0.810252
1000,0.519400,0.450783,0.825553
1500,0.480000,0.404127,0.847186
2000,0.443500,0.419895,0.847437
2500,0.427400,0.390962,0.857094
3000,0.361600,0.439535,0.858160
3500,0.277700,0.403291,0.861953
4000,0.277300,0.376189,0.866531
4500,0.274400,0.362412,0.868443
5000,0.263400,0.387676,0.869447


***** Running Evaluation *****
  Num examples = 31895
  Batch size = 16
Saving model checkpoint to result/pure_test_electra/checkpoint-500
Configuration saved in result/pure_test_electra/checkpoint-500/config.json
Model weights saved in result/pure_test_electra/checkpoint-500/pytorch_model.bin
tokenizer config file saved in result/pure_test_electra/checkpoint-500/tokenizer_config.json
Special tokens file saved in result/pure_test_electra/checkpoint-500/special_tokens_map.json
/home/centos/anaconda3/envs/test_env/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:61: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
***** Running Evaluation *****
  Num examples = 31895
  Batch size = 16
Saving model checkpoint to result/pure_test_electra/checkpoint-1000
Configuration saved in result/pure_test_electra/checkpoint-1000/config.json
Mode

***** Running Evaluation *****
  Num examples = 31895
  Batch size = 16
Saving model checkpoint to result/pure_test_electra/checkpoint-5500
Configuration saved in result/pure_test_electra/checkpoint-5500/config.json
Model weights saved in result/pure_test_electra/checkpoint-5500/pytorch_model.bin
tokenizer config file saved in result/pure_test_electra/checkpoint-5500/tokenizer_config.json
Special tokens file saved in result/pure_test_electra/checkpoint-5500/special_tokens_map.json
Deleting older checkpoint [result/pure_test_electra/checkpoint-3000] due to args.save_total_limit
/home/centos/anaconda3/envs/test_env/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:61: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
***** Running Evaluation *****
  Num examples = 31895
  Batch size = 16
Saving model checkpoint to result/pure_test_e

tokenizer config file saved in result/pure_test_electra/checkpoint-10000/tokenizer_config.json
Special tokens file saved in result/pure_test_electra/checkpoint-10000/special_tokens_map.json
Deleting older checkpoint [result/pure_test_electra/checkpoint-8000] due to args.save_total_limit
/home/centos/anaconda3/envs/test_env/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:61: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
***** Running Evaluation *****
  Num examples = 31895
  Batch size = 16
Saving model checkpoint to result/pure_test_electra/checkpoint-10500
Configuration saved in result/pure_test_electra/checkpoint-10500/config.json
Model weights saved in result/pure_test_electra/checkpoint-10500/pytorch_model.bin
tokenizer config file saved in result/pure_test_electra/checkpoint-10500/tokenizer_config.json
Special tokens fi

/home/centos/anaconda3/envs/test_env/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:61: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
***** Running Evaluation *****
  Num examples = 31895
  Batch size = 16
Saving model checkpoint to result/pure_test_electra/checkpoint-15000
Configuration saved in result/pure_test_electra/checkpoint-15000/config.json
Model weights saved in result/pure_test_electra/checkpoint-15000/pytorch_model.bin
tokenizer config file saved in result/pure_test_electra/checkpoint-15000/tokenizer_config.json
Special tokens file saved in result/pure_test_electra/checkpoint-15000/special_tokens_map.json
Deleting older checkpoint [result/pure_test_electra/checkpoint-13000] due to args.save_total_limit
/home/centos/anaconda3/envs/test_env/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:61: UserWarnin

***** Running Evaluation *****
  Num examples = 31895
  Batch size = 16
Saving model checkpoint to result/pure_test_electra/checkpoint-19500
Configuration saved in result/pure_test_electra/checkpoint-19500/config.json
Model weights saved in result/pure_test_electra/checkpoint-19500/pytorch_model.bin
tokenizer config file saved in result/pure_test_electra/checkpoint-19500/tokenizer_config.json
Special tokens file saved in result/pure_test_electra/checkpoint-19500/special_tokens_map.json
Deleting older checkpoint [result/pure_test_electra/checkpoint-17500] due to args.save_total_limit
/home/centos/anaconda3/envs/test_env/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:61: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from r

In [50]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

Tokenizer_NAME = "tunib/electra-ko-base"
tokenizer = AutoTokenizer.from_pretrained(Tokenizer_NAME)

MODEL_NAME = 'result/pure_test_electra/best_model'
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)
model.resize_token_embeddings(tokenizer.vocab_size)
model.to(device)

print(tokenizer)

loading configuration file config.json from cache at /home/centos/.cache/huggingface/hub/models--tunib--electra-ko-base/snapshots/edfb795c9f667b3c5cb7085ca9112997823ce4e8/config.json
Model config ElectraConfig {
  "_name_or_path": "tunib/electra-ko-base",
  "architectures": [
    "ElectraForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 768,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "summary_activation": "gelu",
  "summary_last_dropout": 0.1,
  "summary_type": "first",
  "summary_use_proj": true,
  "transformers_version": "4.24.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32000
}

loading file vocab.txt from cach

PreTrainedTokenizerFast(name_or_path='tunib/electra-ko-base', vocab_size=32000, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})


In [51]:
test_label = label_to_num(test['label'].values)

tokenized_test = tokenizer(
    list(test['premise']),
    list(test['hypothesis']),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

test_dataset = BERTDataset(tokenized_test, test_label)

print(test_dataset.__len__())
print(test_dataset.__getitem__(1665))
print(tokenizer.decode(test_dataset.__getitem__(6)['input_ids']))

1666
{'input_ids': tensor([    2,   791, 14921, 23745,  6237,   485, 15213,  2137,   204,  8377,
        11304,     3,   791, 14921,  1485,  6020, 11461,  6015, 14521, 13377,
            3,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0]), 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [52]:
dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)

model.eval()
output_pred = []
output_prob = []

for i, data in enumerate(tqdm(dataloader)):
    with torch.no_grad():
        outputs = model(
            input_ids=data['input_ids'].to(device),
            attention_mask=data['attention_mask'].to(device),
            token_type_ids=data['token_type_ids'].to(device)
        )
    logits = outputs[0]
    prob = F.softmax(logits, dim=-1).detach().cpu().numpy()
    logits = logits.detach().cpu().numpy()
    result = np.argmax(logits, axis=-1)

    output_pred.append(result)
    output_prob.append(prob)
  
pred_answer, output_prob = np.concatenate(output_pred).tolist(), np.concatenate(output_prob, axis=0).tolist()
print(pred_answer)

100%|██████████| 105/105 [00:02<00:00, 44.64it/s]

[1, 2, 0, 1, 1, 2, 1, 0, 0, 1, 1, 0, 1, 0, 2, 2, 0, 2, 1, 2, 2, 2, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 2, 2, 1, 0, 1, 0, 1, 2, 0, 2, 2, 2, 1, 2, 0, 1, 0, 1, 1, 1, 2, 0, 2, 2, 2, 1, 0, 0, 2, 2, 2, 2, 1, 2, 2, 2, 1, 0, 2, 0, 0, 2, 2, 0, 2, 2, 1, 2, 2, 1, 1, 0, 1, 2, 1, 0, 2, 1, 2, 0, 1, 0, 0, 1, 1, 0, 2, 2, 0, 2, 0, 2, 0, 2, 2, 1, 1, 1, 2, 1, 0, 2, 0, 0, 1, 1, 2, 1, 1, 1, 2, 0, 1, 0, 1, 1, 2, 0, 2, 1, 0, 2, 0, 1, 2, 1, 1, 2, 2, 2, 0, 2, 0, 0, 1, 2, 2, 1, 0, 1, 2, 0, 0, 0, 1, 2, 1, 2, 0, 1, 0, 2, 1, 2, 0, 2, 0, 2, 2, 1, 0, 2, 1, 2, 2, 1, 1, 1, 2, 2, 1, 2, 2, 1, 2, 0, 1, 2, 0, 1, 0, 2, 0, 0, 2, 0, 2, 2, 2, 0, 1, 1, 0, 0, 2, 0, 2, 1, 1, 1, 2, 1, 1, 1, 0, 1, 1, 2, 2, 0, 1, 2, 1, 1, 2, 1, 1, 0, 2, 1, 0, 0, 2, 2, 1, 0, 0, 0, 1, 2, 2, 2, 0, 2, 2, 1, 1, 0, 1, 2, 0, 1, 2, 1, 1, 0, 0, 0, 0, 2, 2, 2, 2, 1, 0, 2, 0, 1, 2, 0, 0, 0, 1, 1, 1, 2, 2, 2, 0, 0, 0, 1, 0, 1, 1, 0, 2, 0, 1, 0, 1, 0, 1, 1, 1, 2, 0, 2, 0, 2, 0, 1, 0, 2, 1, 0, 0, 2, 1, 1, 2, 2, 2, 2, 2, 0, 0, 0, 2, 2, 2, 1, 0, 1, 1, 1, 1, 2, 0, 2, 

In [53]:
def num_to_label(label):
    label_dict = {0: "entailment", 1: "contradiction", 2: "neutral"}
    str_label = []

    for i, v in enumerate(label):
        str_label.append([i,label_dict[v]])
    
    return str_label

answer = num_to_label(pred_answer)
print(answer)

[[0, 'contradiction'], [1, 'neutral'], [2, 'entailment'], [3, 'contradiction'], [4, 'contradiction'], [5, 'neutral'], [6, 'contradiction'], [7, 'entailment'], [8, 'entailment'], [9, 'contradiction'], [10, 'contradiction'], [11, 'entailment'], [12, 'contradiction'], [13, 'entailment'], [14, 'neutral'], [15, 'neutral'], [16, 'entailment'], [17, 'neutral'], [18, 'contradiction'], [19, 'neutral'], [20, 'neutral'], [21, 'neutral'], [22, 'contradiction'], [23, 'entailment'], [24, 'contradiction'], [25, 'contradiction'], [26, 'entailment'], [27, 'entailment'], [28, 'entailment'], [29, 'entailment'], [30, 'contradiction'], [31, 'entailment'], [32, 'contradiction'], [33, 'neutral'], [34, 'neutral'], [35, 'contradiction'], [36, 'entailment'], [37, 'contradiction'], [38, 'entailment'], [39, 'contradiction'], [40, 'neutral'], [41, 'entailment'], [42, 'neutral'], [43, 'neutral'], [44, 'neutral'], [45, 'contradiction'], [46, 'neutral'], [47, 'entailment'], [48, 'contradiction'], [49, 'entailment'], 

In [54]:
df = pd.DataFrame(answer, columns=['index', 'label'])

df.to_csv('pure_test_electra.csv', index=False)

print(df)

      index          label
0         0  contradiction
1         1        neutral
2         2     entailment
3         3  contradiction
4         4  contradiction
...     ...            ...
1661   1661        neutral
1662   1662     entailment
1663   1663        neutral
1664   1664        neutral
1665   1665     entailment

[1666 rows x 2 columns]
